# Uniview module showing SNe in the SDSS galaxies on the LSST discovery rate

LSST cadence data from [PLAsTiCC Classification Challenge](https://zenodo.org/record/2539456#.XX-1apNKhTY).  Using all the SDSS spiral galaxies in our data set, this covers about 1/4 year (2023) of LSST observations.

*Aaron Geller, 2019*

### Imports and function definitions

In [7]:
#This directory contains all the data needed for the module. It should be in the same directory as the notebook
dataFolder = "data" 
import sys, os, shutil, errno, string, urllib


sys.path.append(( os.path.abspath( os.path.join(os.path.realpath("__file__"),  os.pardir, os.pardir) )))
import uvmodlib.v1 as uvmod

### USES Conf Template 

In [34]:
Template = """mesh
{  
    data SNdata ./modules/SDSS_LSST_SNe/SDSSLSSTSNe.raw

    dataHints SNdata disablePostprocessing
    
    cullRadius 10000.0
    glslVersion 330
    
    propertyCollection 
    {        
        __objectName__
        {           

            vec1f radiusScale 6 | public | desc "point size scale" | widget slider | range 0 10
            vec1f SNduration 0.0001 | public | desc "duration of SN light curve" | widget slider | range 0 0.01
            vec1f SNangleMax 1.5 | public | desc "max angular size of SN circle" | widget slider | range 0 30
            vec1f SNangleMin 0.1 | public | desc "min angular size of SN circle" | widget slider | range 0 3

        }
    }
    
    #############
    pass
    {
        useDataObject SNdata
        shader
        {
            type defaultMeshShader
            {
                geometryShader   ./modules/SDSS_LSST_SNe/SNvtime.gs
                vertexShader ./modules/SDSS_LSST_SNe/SNvtime.vs
                fragmentShader   ./modules/SDSS_LSST_SNe/SNvtime.fs
                
                stateManagerVar __objectName__.radiusScale  radiusScale
                stateManagerVar __objectName__.SNduration  SNduration
                stateManagerVar __objectName__.SNangleMax  SNangleMax
                stateManagerVar __objectName__.SNangleMin  SNangleMin
                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }
 

    
}"""

### SN class

In [35]:
class SNClass():
    def __init__(self, object):
        self.object = object
        uvmod.Utility.ensurerelativepathexsists("SNvtime.gs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("SNvtime.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("SNvtime.fs",dataFolder)
        self.cr = 1000
        self.Scale = 1

    def generatemod(self):
        self.object.setgeometry(self.object.name+"Mesh.usesconf")
        return self.object.generatemod()
    def generatefiles(self, absOutDir, relOutDir):
        fileName = self.object.name+"Mesh.usesconf"
        s = string.Template(Template)
        f = open(absOutDir+"\\"+fileName, 'w')
        if f:
            f.write(s.substitute(folderName = relOutDir,
                                 cr = self.cr,
                                 Scale = self.Scale
                                 ))
            f.close()
        uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, absOutDir)

### Object Instantiation

In [36]:
model = SNClass(uvmod.OrbitalObject())
scene = uvmod.Scene()
parentScene = uvmod.Scene()
modinfo = uvmod.ModuleInformation()
generator = uvmod.Generator()

### Specify Settings and generate the module

In [37]:
scene.setname("SDSS_LSST_SNe")
scene.setparent("Extragalactic")
scene.setunit(3.09e22) #km to Mpc


model.object.setcameraradius(1e-3) #near plane clipping?
model.object.setcoord(scene.name)
model.object.setname("SDSS_LSST_SNe")
model.object.setguiname("/KavliLecture/Zooniverse/SDSS_LSST_SNe")
model.object.settargetradius(100)
model.object.showatstartup(False)
model.cr = 1e4 #far plane clipping?

modinfo.setname("SDSS_LSST_SNe")
modinfo.setauthor("Aaron Geller<sup>1</sup>, Laura Trouille<sup>1</sup>, Chris Lintott<sup>2</sup> <br />(1)Adler Planetarium,<br />(2)Oxford University,")
modinfo.cleardependencies()
modinfo.setdesc("Uniview module for showing SNe in the SDSS spiral galaxies at the expected LSST discovery rate, over 1/4 year 2023.  Rate is taken from the PLAsTiCC Classification Challenge : https://zenodo.org/record/2539456#.XX-1apNKhTY .")
#modinfo.setthumbnail("data/SNvTimeSnapshot.png")
modinfo.setversion("1.0")

generator.generate("SDSS_LSST_SNe",[scene],[model],modinfo)
uvmod.Utility.senduvcommand(model.object.name+".reload")


## To reload

In [21]:
from config import Settings

In [23]:
uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, Settings.uvcustommodulelocation+'\\'+model.object.name)
uvmod.Utility.senduvcommand(model.object.name+".reload")